# Merge PAO public data files in one file for the analysis

A good part of this code is copied from the PAO public data release. Reference TO BE ADDED


In [1]:
import pandas as pd
import numpy as np
import datetime

In [2]:
# Data loading, encapsulated to make it less installation and OS dependant
import os.path
from zipfile import ZipFile
def AugerLoad(fdir,file):
    """
    Loads a file from the auger open data release. Can be either in the local directory,
    in the parent directory or in the augeropendata directory.
    File is identified by it directory *fdir* and filename *file* and can be found in the directory
    or in a zip file.
    """
    for loc in [".","..","augeropendata"]:
        fname=os.path.join(loc,fdir,file)
        if os.path.isfile(fname):
            print(f'Opening: {fname}. The file is unzipped')
            return open(fname)
        zname=os.path.join(loc,fdir+".zip")
        if os.path.isfile(zname):
            with ZipFile(zname) as myzip:
                print(f'Opening: {fname}. The file is still zipped')
                return myzip.open(os.path.join(fdir,file))

In [9]:
# open vertical and inclined events files

sd1500_data = pd.read_csv(AugerLoad(".","dataSummarySD1500.csv"))
sdinclined_data = pd.read_csv(AugerLoad(".","dataSummaryInclined.csv"))

Opening: ././dataSummarySD1500.csv. The file is unzipped
Opening: ././dataSummaryInclined.csv. The file is unzipped


In [10]:
#drop duplicates

print('Number of events before dropping the duplicates:')
print('SD1500:', len(sd1500_data))
print('Inclined:', len(sdinclined_data))

sd1500_data = sd1500_data.drop_duplicates('id')
sdinclined_data = sdinclined_data.drop_duplicates('id')

print('Number of events after dropping the duplicates:')
print('SD1500:', len(sd1500_data))
print('Inclined:', len(sdinclined_data))

Number of events before dropping the duplicates:
SD1500: 24319
Inclined: 2355
Number of events after dropping the duplicates:
SD1500: 24285
Inclined: 2355


In [11]:
def get_datetime(data):
    event_id = data.id

    # extract date and time from id using integer division // and modulus
    year_factor = 10**10
    day_factor =  10**7
    day_modulus = 10**3
    second_factor = 100
    second_modulus = 10**5

    years = event_id // year_factor
    days = event_id // day_factor % day_modulus
    seconds = event_id // second_factor % second_modulus
    years = years + 2000

    # generate a 'datetime' variable
    # NB: the Auger day starts at noon UTC
    date = [datetime.datetime(year, 1, 1, 12, tzinfo=datetime.timezone.utc) # initialise data at the beginning of the year
            + datetime.timedelta(days=day - 1, seconds=second - 1)  #adds the additional days as read from the dataframe
            for year, day, second in zip(years, days, seconds)]
    # add the column 'date' to the dataframe
    data['date']=date

    return data

In [12]:
# add datetime to the dataframes

sd1500_data = get_datetime(sd1500_data)
sdinclined_data = get_datetime(sdinclined_data)

In [14]:
# now we also load the UHECR catalog

uhecr_data = pd.read_csv(AugerLoad(".","auger_catalogSD.csv"))

Opening: ././auger_catalogSD.csv. The file is unzipped


In [15]:
#add datetime also to UHECR catalog

id_array = uhecr_data.id

date = []
year = []
month = []
day = []
for i in id_array:
    date.append(i[3:])
    year.append(int('20'+i[3:5]))
    month.append(int(i[5:7]))
    day.append(int(i[7:9]))

utc_time = [datetime.datetime.fromtimestamp(t, tz=datetime.timezone.utc) for t in uhecr_data.utctime.values]

uhecr_data['date'] = utc_time

In [18]:
# now we select only the columns we are interested in and we make sure thay have the same name

print('Columns in the vertical events dataframe:', sd1500_data.columns)

print('Columns in the inclined dataframe:', sdinclined_data.columns)

print('Columns in the UHECR catalog dataframe:', uhecr_data.columns)

Columns in the vertical events dataframe: Index(['id', 'sdid', 'gpstime', 'sd1500', 'multiEye', 'sd_gpsnanotime',
       'sd_theta', 'sd_dtheta', 'sd_phi', 'sd_dphi', 'sd_energy', 'sd_denergy',
       'sd_l', 'sd_b', 'sd_ra', 'sd_dec', 'sd_x', 'sd_dx', 'sd_y', 'sd_dy',
       'sd_z', 'sd_easting', 'sd_northing', 'sd_altitude', 'sd_R', 'sd_dR',
       'sd_s1000', 'sd_ds1000', 'sd_s38', 'sd_gcorr', 'sd_wcorr', 'sd_beta',
       'sd_gamma', 'sd_chi2', 'sd_ndf', 'sd_geochi2', 'sd_geondf', 'sd_nbstat',
       'fd_id', 'fd_gpsnanotime', 'fd_hdSpectrumEye', 'fd_hdCalibEye',
       'fd_hdXmaxEye', 'fd_theta', 'fd_dtheta', 'fd_phi', 'fd_dphi', 'fd_l',
       'fd_b', 'fd_ra', 'fd_dec', 'fd_totalEnergy', 'fd_dtotalEnergy',
       'fd_calEnergy', 'fd_dcalEnergy', 'fd_xmax', 'fd_dxmax', 'fd_heightXmax',
       'fd_distXmax', 'fd_dEdXmax', 'fd_ddEdXmax', 'fd_x', 'fd_dx', 'fd_y',
       'fd_dy', 'fd_z', 'fd_easting', 'fd_northing', 'fd_altitude',
       'fd_cherenkovFraction', 'fd_minViewAngle', 'fd_

In [27]:
# select only the columns we are interested in
cols_sd = ['id', 'date', 'sd_energy', 'sd_denergy', 'sd_ra', 'sd_dec'  ]

cols_UHECR = ['id', 'date', 'energy', 'denergy', 'ra', 'dec']

In [31]:
sd1500_analysis_cols = sd1500_data[cols_sd].rename(columns = {'sd_energy': 'energy', 'sd_denergy': 'denergy','sd_ra': 'ra', 'sd_dec': 'dec'})
sdinclined_analysis_cols = sdinclined_data[cols_sd].rename(columns = {'sd_energy': 'energy', 'sd_denergy': 'denergy','sd_ra': 'ra', 'sd_dec': 'dec'})
uhecr_data_analysis_cols = uhecr_data[cols_UHECR]

In [34]:
#now let's delect the overlapping events
overlap_sd1500 = np.intersect1d(sd1500_analysis_cols.date.values, uhecr_data_analysis_cols.date.values)

mask = np.isin(sd1500_analysis_cols.date.values, overlap_sd1500)

sd1500_analysis_cols = sd1500_analysis_cols[~mask]

In [35]:
#now we merge the events
all_events_merged = pd.concat([sd1500_analysis_cols, sdinclined_analysis_cols, uhecr_data_analysis_cols])

In [39]:
#save the file!
cr_events = all_events_merged.to_records()

np.save('cr_events.npy', cr_events)